# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:
hands = ['QTx.Jx.T8xx.QTxx','KJ8x.9xxx.KJ9.9x','A9x.KQ8xx.Q.KJ8x','xxx.AT.Axxxx.Axx']

In [4]:
bidder_bots = [BotBid([True, False], hand, models, sampler, i, 2, False) for i, hand in enumerate(hands)]

In [5]:
auction = ["PAD_START"] 

turn_i = 1 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    if step== 2:
        bid.bid='2D'
    if step== 7:
        bid.bid='PASS'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [10.6, 11.1, 10.9], 'shape': [3.0, 3.0, 3.2, 3.7, 3.0, 3.0, 3.2, 3.5, 3.0, 3.0, 3.3, 3.6]}
{'bid': '1H', 'who': 'NN', 'candidates': [{'call': '1H', 'insta_score': 0.998}], 'hcp': [9.0, 9.5, 6.1], 'shape': [3.2, 2.5, 4.1, 3.1, 3.2, 2.7, 3.9, 3.0, 3.1, 2.7, 4.0, 3.1]}
{'bid': '2D', 'who': 'NN', 'candidates': [{'call': 'X', 'insta_score': 0.968}], 'hcp': [7.9, 6.6, 13.0], 'shape': [3.7, 2.7, 2.9, 3.6, 3.7, 2.7, 3.0, 3.6, 2.9, 5.4, 2.0, 2.8]}
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [7.2, 14.6, 13.1], 'shape': [4.3, 2.9, 2.2, 3.5, 3.1, 5.3, 1.7, 2.9, 2.7, 2.5, 5.3, 2.5]}
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 0.89}], 'hcp': [13.3, 13.3, 5.1], 'shape': [2.8, 5.2, 1.7, 3.3, 2.5, 2.0, 5.5, 3.0, 3.8, 1.8, 2.9, 4.5]}
{'bid': 'X', 'who': 'NN', 'candidates': [{'call': 'X', 'insta_score': 0.975}], 'hcp': [13.6, 5.5, 6.0], 's